In [1]:
import pandas as pd

from langchain_dartmouth.llms import ChatDartmouth

In [ ]:
docs = pd.read_csv("data/raw/2024_COFE_SS_sample_text.csv")
question = docs.loc[0, "outcometxt"]
question

In [ ]:
docs = docs.iloc[1:]  # The first row is a comment on the column contents
docs = docs.dropna(subset="outcometxt")
responses = docs["outcometxt"].to_list()
responses[:5]

In [ ]:
llm = ChatDartmouth(
    model_name="llama-3-1-8b-instruct",
    temperature=0,
    seed=42,
    max_tokens=60_000,
)

example_response = (
    "Dartmouth has been a transformative experience that's shaped my academic, "
    "personal, and professional growth. I've gained invaluable knowledge and "
    "skills through rigorous courses in econ and international relations, which "
    "have helped me develop a nuanced perspective on global issues. However, I've "
    "also struggled with the intense academic pressure and lack of diversity in "
    "certain departments."
)

example_output = """{
    "response" : <original response>,
    "topics": [
        {"name": "academic and personal growth", "sentiment": "positive"},
        {"name": "academic rigor", "sentiment": "positive"},
        {"name": "academic pressure", "sentiment": "negative"},
        {"name": "lack of diversity", "sentiment": "negative"},
    ]
}"""

prompt = (
    "The following are survey responses from students in their senior year at Dartmouth College.\n"
    f"The prompt was: '{question}'\n"
    "Identify the topics mentioned in each response, "
    "as well as the sentiments expressed towards those topics. "
    "Format your response in valid JSON. Here is an example: \n\n"
    f"Response: \n'{example_response}'\n\n"
    f"Output: \n{example_output}\n\n"
    "Here are the responses to process, separated by '\\n------\\n':\n\n"
    f"{'\n------\n'.join(responses)}"
)

print(prompt[:1500])

In [ ]:
response = llm.invoke(prompt)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

parser.invoke(response)